In [ ]:
!pip install gensim

In [0]:
import os
import re
import pickle

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import SpatialDropout1D, Conv1D, MaxPooling1D
from gensim.models import Word2Vec
from keras.utils import np_utils
from keras.preprocessing import sequence
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
import seaborn as sns
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib import colors

from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from google.colab import files
 

pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
w2v_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz")
hh_data_path = os.path.join("/content/gdrive/My Drive/", "Colab Notebooks/hh_dataset.csv")

In [0]:
dataset_hh = pd.read_csv(hh_data_path, sep="\t")
dataset_hh = dataset_hh.loc[dataset_hh["lang_text"] == "russian"]
#dataset_hh.drop(dataset_hh.index[dataset_hh["lang_text"] == "english"], inplace=True)
dataset_hh.reset_index(drop=True, inplace=True)
dataset_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25223 entries, 0 to 25222
Data columns (total 26 columns):
id                            25223 non-null int64
lang_text                     25223 non-null object
lang_title                    25223 non-null object
profarea_names                25223 non-null object
requirement                   24322 non-null object
requirement_lemmas            24321 non-null object
requirement_lemmas_tags       24321 non-null object
requirement_norm              24321 non-null object
requirement_tokens            24321 non-null object
responsibility                23890 non-null object
responsibility_lemmas         23889 non-null object
responsibility_lemmas_tags    23889 non-null object
responsibility_norm           23889 non-null object
responsibility_tokens         23889 non-null object
specializations               25223 non-null object
text                          25223 non-null object
text_lemmas                   25223 non-null object
text_lem

In [0]:
dataset_hh.drop_duplicates(["text"], inplace=True)
print("Final size of dataset =", len(dataset_hh))
dataset_hh.reset_index(drop=True, inplace=True)
dataset_hh.info()

Final size of dataset = 17250
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 26 columns):
id                            17250 non-null int64
lang_text                     17250 non-null object
lang_title                    17250 non-null object
profarea_names                17250 non-null object
requirement                   16763 non-null object
requirement_lemmas            16762 non-null object
requirement_lemmas_tags       16762 non-null object
requirement_norm              16762 non-null object
requirement_tokens            16762 non-null object
responsibility                16481 non-null object
responsibility_lemmas         16480 non-null object
responsibility_lemmas_tags    16480 non-null object
responsibility_norm           16480 non-null object
responsibility_tokens         16480 non-null object
specializations               17250 non-null object
text                          17250 non-null object
text_lemmas                   

In [0]:
dataset_hh.head()

,id,lang_text,lang_title,profarea_names,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,responsibility_norm,responsibility_tokens,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url
0,28109665,russian,russian,"['Административный персонал', 'Продажи']",Высшее образование. Опыт работы в организации ...,высший образование опыт работа организация кли...,высший_ADJ образование_NOUN опыт_NOUN работа_N...,высш образован оп работ организац клиентск сер...,высшее образование опыт работы организации кли...,Организация обучения сотрудников подразделения...,организация обучение сотрудник подразделение о...,организация_NOUN обучение_NOUN сотрудник_NOUN ...,организац обучен сотрудник подразделен организ...,организация обучения сотрудников подразделения...,"['Ресепшен', 'Менеджер по работе с клиентами']",<p>Группа компаний «Три-З» занимает лидирующие...,группа компания занимать лидировать позиция ро...,группа_NOUN компания_NOUN занимать_VERB лидиро...,групп компан занима лидир позиц российск рынк ...,группа компаний занимает лидирующие позиции ро...,Начальник отдела клиентского сервиса,начальник отдел клиентский сервис,начальник_NOUN отдел_NOUN клиентский_ADJ серви...,начальник отдел клиентск сервис,начальник отдела клиентского сервиса,https://api.hh.ru/vacancies/28109665?host=hh.ua
1,29258242,russian,russian,"['Закупки', 'Закупки', 'Закупки', 'Продажи', '...",Нам нужен человек обязательно с опытом работы ...,мы нужный человек обязательно опыт работа мага...,мы_PRON нужный_ADJ человек_NOUN обязательно_AD...,нам нуж человек обязательн опыт работ магазин ...,нам нужен человек обязательно опытом работы ма...,Скупать товары народного потребления у входящи...,скупать товар народный потребление входить кли...,скупать_VERB товар_NOUN народный_ADJ потреблен...,скупа товар народн потреблен входя клиент прод...,скупать товары народного потребления входящих ...,"['Электроника, фото, видео', 'Электротехническ...","<p>Привет, мы – компания СКУПКА.<br />Мы скупа...",привет компания скупавать продавать различный ...,привет_VERB компания_NOUN скупавать_VERB прода...,привет компан скупа прода различн товар электр...,привет компания скупаем продаем различные това...,Менеджер по продажам,менеджер продажа,менеджер_NOUN продажа_NOUN,менеджер продаж,менеджер продажам,https://api.hh.ru/vacancies/29258242?host=hh.ua
2,28944166,russian,russian,"['Домашний персонал', 'Домашний персонал', 'На...",Опыт работы педагогом в детских дошкольных учр...,опыт работа педагог детский дошкольный учреждение,опыт_NOUN работа_NOUN педагог_NOUN детский_ADJ...,оп работ педагог детск дошкольн учрежден,опыт работы педагогом детских дошкольных учреж...,Проведение занятий с детьми от 8 мес. до 7 лет...,проведение занятие ребенок месяц лета методика...,проведение_NOUN занятие_NOUN ребенок_NUM месяц...,проведен занят детьм мес лет методик консульти...,проведение занятий детьми мес лет методикам ко...,"['Воспитатель, Гувернантка, Няня', 'Репетитор'...",<p>«Бэби-клуб Матвеевский» приглашает педагога...,матвеевский приглашать педагог ранний развитие...,матвеевский_NOUN приглашать_VERB педагог_NOUN ...,матвеевск приглаша педагог ран развит бережн р...,матвеевский приглашает педагога раннему развит...,Специалист (педагог) по раннему развитию,специалист педагог ранний развитие,специалист_NOUN педагог_NOUN ранний_ADJ развит...,специалист педагог ран развит,специалист педагог раннему развитию,https://api.hh.ru/vacancies/28944166?host=care...
3,29258121,russian,russian,"['Наука, образование', 'Наука, образование', '...","Творческий подход, командный дух, инициативу и...",творческий подход командный дух инициатива улы...,творческий_ADJ подход_NOUN командный_ADJ дух_N...,творческ подход командн дух инициатив улыбк тр...,творческий подход командный дух инициативу улы...,Участие в общественно-маркетинго

### Prapare data to rnn

In [0]:
vectorizer = KeyedVectors.load_word2vec_format(w2v_path, binary=False)


In [0]:
files.upload()

Saving vectorizer_tfidf.dat to vectorizer_tfidf (1).dat


In [ ]:
with open("vectorizer_tfidf (1).dat", "rb") as inf:
  vectorizer_tfidf = pickle.load(inf)

In [0]:
%%time
corpus = [text for text in dataset_hh.loc[:, "text_normalized"]]

vectorizer_tfidf = TfidfVectorizer(min_df=1, ngram_range=(1, 2),  lowercase=False)
vectorizer_tfidf.fit(corpus)
#ngram_range=(1,)

CPU times: user 6.64 s, sys: 53.5 ms, total: 6.69 s
Wall time: 6.71 s


In [0]:
corpus = [text for text in dataset_hh.loc[:, "text_normalized"]]

In [0]:

print(len(vectorizer_tfidf.get_feature_names()))
print(vectorizer_tfidf.get_feature_names()[10001])

1109782
area order


In [0]:
with open("vectorizer_tfidf_hh.dat", "wb") as ouf:
  pickle.dump(vectorizer_tfidf, ouf)

In [0]:
files.download("vectorizer_tfidf_hh.dat")

### Create labels

In [0]:
labels = []
for i, row in dataset_hh.iterrows():
    specs = row["profarea_names"].lower()
    specs = specs.split("', ") 
    for spec in specs: 
        spec = re.sub('[\[\'\]]', '', spec)
        labels.append(spec.strip())

labels = list(set(labels))

print("Labels number:", len(labels))
print("Labels:", labels)

Labels number: 28
Labels: ['информационные технологии, интернет, телеком', 'строительство, недвижимость', 'страхование', 'спортивные клубы, фитнес, салоны красоты', 'высший менеджмент', 'маркетинг, реклама, pr', 'медицина, фармацевтика', 'консультирование', 'бухгалтерия, управленческий учет, финансы предприятия', 'добыча сырья', 'государственная служба, некоммерческие организации', 'рабочий персонал', 'наука, образование', 'закупки', 'автомобильный бизнес', 'управление персоналом, тренинги', 'транспорт, логистика', 'производство', 'туризм, гостиницы, рестораны', 'продажи', 'юристы', 'банки, инвестиции, лизинг', 'административный персонал', 'начало карьеры, студенты', 'домашний персонал', 'безопасность', 'инсталляция и сервис', 'искусство, развлечения, масс-медиа']


In [0]:
labels = ['информационные технологии, интернет, телеком', 'строительство, недвижимость', 'спортивные клубы, фитнес, салоны красоты', 'маркетинг, реклама, pr', 'медицина, фармацевтика', 'консультирование', 'бухгалтерия, управленческий учет, финансы предприятия', 'рабочий персонал', 'наука, образование', 'автомобильный бизнес', 'управление персоналом, тренинги', 'транспорт, логистика', 'производство', 'туризм, гостиницы, рестораны', 'продажи', 'юристы', 'банки, инвестиции, лизинг', 'административный персонал', 'начало карьеры, студенты', 'искусство, развлечения, масс-медиа']

In [0]:
def vectorize_text(text, vectorizer, max_len):
    """
    :param str text: normalized text
    :param Word2Vec vectorizer:
    :param int max_len:
    :return np.array text_vect: of shape like (1, max_len, 100)
    """
    text_vect = []
    words = text.split(" ")
    for word in words:
        try:
            word_vect = vectorizer.word_vec(word)
            text_vect.append(word_vect)
        except KeyError:
            None
      
    np.array(text_vect)
    text_vect = np.reshape(text_vect, (1, -1, 100))
    text_vect = sequence.pad_sequences(text_vect, maxlen=max_len, dtype='float')
            
    return text_vect
    

### Prepare X and Y using tf-idf


In [0]:
%%time
# Try using tf-idf to classification
X = vectorizer_tfidf.transform(corpus)
X.shape

CPU times: user 4.66 s, sys: 11.7 ms, total: 4.67 s
Wall time: 4.69 s


In [0]:
n_classes = len(labels)
n_examples = len(dataset_hh)
Y = np.zeros((n_examples, n_classes))

for i, row in dataset_hh.iterrows():     
    specs = row["profarea_names"].lower() 
    specs = specs.split("', ") 
    for spec in specs: 
        spec = re.sub('[\[\'\]]', '', spec)
        if spec in labels:
          pos = labels.index(spec.strip())
        Y[i][pos] = 1
        
assert(X.shape[0] == Y.shape[0])

### Prepare X and Y using mean word2vec for document weighted by tf-idf

In [ ]:
%%time
dict_tfidf = vectorizer_tfidf.get_feature_names()

n_classes = len(labels)
n_examples = len(dataset_hh)
X = []
Y = np.zeros((n_examples, n_classes))
for i, row in dataset_hh.iterrows():
    X.append([])
    text = row["text_lemmas_tags"]
    words = text.split(" ")
    text_tfidf = vectorizer_tfidf.transform([text])
    for word in words:
        try:
            word_vec = vectorizer.word_vec(word)
            if word in dict_tfidf:
              pos = dict_tfidf.index(word)
              weight = text_tfidf[0, pos]
              word_vec = word_vec * weight
            
            X[i].append(word_vec)
        except KeyError:
            None
     
    
    specs = row["profarea_names"].lower() 
    specs = specs.split("', ") 
    for spec in specs: 
        spec = re.sub('[\[\'\]]', '', spec)
        pos = labels.index(spec.strip())
        Y[i][pos] = 1
        
    if i % 100 == 0:
      print("Document number -", i)
        
assert(len(X) == len(Y))


In [0]:
print(len(X))
print(Y.shape)

10521
(10521, 28)


### Prepare X and Y using word2vec 

In [0]:

n_classes = len(labels)
n_examples = len(dataset_hh)
X = []
Y = np.zeros((n_examples, n_classes))
for i, row in dataset_hh.iterrows():
    X.append([])
    words = row["text_lemmas_tags"].split(" ")
    for word in words:
        try:
            word_vec = vectorizer.word_vec(word)
            X[i].append(word_vec)
        except KeyError:
            None
     
    
    specs = row["profarea_names"].lower() 
    specs = specs.split("', ") 
    for spec in specs: 
        spec = re.sub('[\[\'\]]', '', spec)
        pos = labels.index(spec.strip())
        Y[i][pos] = 1
        
assert(len(X) == len(Y))


In [0]:
max_len = max([len(i) for i in X])
print("Max len =", max_len)
max_len = 300
print("Max len edited=", max_len)

Max len = 777
Max len edited= 300


In [0]:
#X = sequence.pad_sequences(X, maxlen=max_len, dtype='float')

In [0]:
print(len(X))
print(Y.shape)

22556
(22556, 28)


In [0]:
def get_classes(y_one_hot, labels, threshold=0.5):
    """
    :param np.array y_one_hot: shape like (1, n_classes)
    :param list of str labels:
    :return list of str classes: 
    """
    classes = []
    y_rounded = np.zeros(y_one_hot.shape)
    y_rounded[y_one_hot > threshold] = 1
    for i in range(len(labels)):
        if y_rounded[0][i] == 1:
            classes.append(labels[i])
            
    return classes
        

### Calculate counts for labels

In [0]:
counts = []
for i in range(len(labels)):
    print(i, ' Label {} appears {} times'.format(labels[i], np.sum(Y[:,i])))
    counts.append(np.sum(Y[:,i]))
print('###')
all_sum_y = np.sum(Y, axis=1)
print('As a mean {} species appear on each recording (standard deviation {})'.format(np.mean(all_sum_y), np.std(all_sum_y)))

0  Label информационные технологии, интернет, телеком appears 2463.0 times
1  Label строительство, недвижимость appears 1276.0 times
2  Label страхование appears 146.0 times
3  Label спортивные клубы, фитнес, салоны красоты appears 416.0 times
4  Label высший менеджмент appears 288.0 times
5  Label маркетинг, реклама, pr appears 1528.0 times
6  Label медицина, фармацевтика appears 777.0 times
7  Label консультирование appears 849.0 times
8  Label бухгалтерия, управленческий учет, финансы предприятия appears 1006.0 times
9  Label добыча сырья appears 49.0 times
10  Label государственная служба, некоммерческие организации appears 44.0 times
11  Label рабочий персонал appears 904.0 times
12  Label наука, образование appears 322.0 times
13  Label закупки appears 235.0 times
14  Label автомобильный бизнес appears 613.0 times
15  Label управление персоналом, тренинги appears 502.0 times
16  Label транспорт, логистика appears 1123.0 times
17  Label производство appears 1100.0 times
18  Label 

In [0]:
indexes = []
labels_new = []
counts_new = []

for i in range(len(labels)):
    if counts[i] > 300:
        labels_new.append(labels[i])
        counts_new.append(counts[i])
        indexes.append(i)
        

In [0]:
print(len(labels_new))
print(labels_new)
print()
print(counts_new)

20
['информационные технологии, интернет, телеком', 'строительство, недвижимость', 'спортивные клубы, фитнес, салоны красоты', 'маркетинг, реклама, pr', 'медицина, фармацевтика', 'консультирование', 'бухгалтерия, управленческий учет, финансы предприятия', 'рабочий персонал', 'наука, образование', 'автомобильный бизнес', 'управление персоналом, тренинги', 'транспорт, логистика', 'производство', 'туризм, гостиницы, рестораны', 'продажи', 'юристы', 'банки, инвестиции, лизинг', 'административный персонал', 'начало карьеры, студенты', 'искусство, развлечения, масс-медиа']

[2463.0, 1276.0, 416.0, 1528.0, 777.0, 849.0, 1006.0, 904.0, 322.0, 613.0, 502.0, 1123.0, 1100.0, 1457.0, 6373.0, 324.0, 1141.0, 2525.0, 4356.0, 430.0]


In [0]:
Y = Y[:, indexes]
labels = labels_new
counts = counts_new

### Vectorize document from siquence of words to one vector

In [0]:
X = [np.mean(i, axis=0) for i in X]
X = np.array(X)
print(X.shape)
print(Y.shape)

(22556, 300)
(22556, 20)


In [0]:
"""w2v_weighted_data = {"X": X, "Y": Y, "labels": labels, "counts": counts}

with open("w2v_weighted_tfidt.dat", "wb") as ouf:
  pickle.dump(w2v_weighted_data, ouf)
  
  
files.download("w2v_weighted_tfidt.dat")
  """

### Data split

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=100)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(12075, 1109782)
(5175, 1109782)
(12075, 20)
(5175, 20)


### 1 vs All classification

In [0]:
classes_weights = []
max_count = max(counts_new)
for i in range(len(counts_new)):
    weight = max_count / counts_new[i] 
    classes_weights.append(weight)

classes_weights = [classes_weights]  
print(classes_weights) 
print(len(classes_weights))

[[2.5874949248883476, 4.994514106583072, 15.319711538461538, 4.170811518324608, 8.202059202059202, 7.506478209658422, 6.334990059642147, 7.049778761061947, 19.79192546583851, 10.396411092985318, 12.695219123505977, 5.674977738201247, 5.793636363636364, 4.374056280027454, 1.0, 19.669753086419753, 5.585451358457494, 2.523960396039604, 1.4630394857667586, 14.820930232558139]]
1


In [0]:
%%time
classif_list = []
train_score_list, test_score_list = [], []

# Training
for i in range(len(labels)):
  num_pos = np.sum(Y_train[:, i])
  num_neg = len(Y_train[:, i]) - num_pos
  sample_weights = np.ones(Y_train[:, i].shape)
  sample_weights[Y_train[:, i] == 1] = num_neg / num_pos * 1
  
  y_train_for_label = Y_train[:, i]
  new_classifier = LogisticRegression()
  #new_classifier = RandomForestClassifier(n_estimators=80, max_depth=5)
  #new_classifier = GradientBoostingClassifier(n_estimators=80, max_depth=3)
  #new_classifier = GaussianNB()
  #new_classifier = SVC()
  #new_classifier = MLPClassifier()
  new_classifier.fit(X_train, y_train_for_label, sample_weight=sample_weights)
  classif_list.append(new_classifier)
  #sample_weight=sample_weights
  print("Training for class ", i, labels[i], "... Weight for class =", num_neg / num_pos)

Training for class  0 информационные технологии, интернет, телеком ... Weight for class = 5.825890333521763
Training for class  1 строительство, недвижимость ... Weight for class = 12.298458149779735
Training for class  2 спортивные клубы, фитнес, салоны красоты ... Weight for class = 40.07142857142857
Training for class  3 маркетинг, реклама, pr ... Weight for class = 10.2012987012987
Training for class  4 медицина, фармацевтика ... Weight for class = 21.527985074626866
Training for class  5 консультирование ... Weight for class = 19.024875621890548
Training for class  6 бухгалтерия, управленческий учет, финансы предприятия ... Weight for class = 15.473396998635744
Training for class  7 рабочий персонал ... Weight for class = 17.295454545454547
Training for class  8 наука, образование ... Weight for class = 49.52301255230125
Training for class  9 автомобильный бизнес ... Weight for class = 27.68171021377672
Training for class  10 управление персоналом, тренинги ... Weight for class = 

In [0]:
# Test & display results
for i in range(len(labels)):
    classif = classif_list[i]
    #train_score = float('{0:.3f}'.format(classif.score(X_train, Y_train[:,i])))
    #test_score = float('{0:.3f}'.format(classif.score(X_test, Y_test[:,i])))
    predict = classif.predict(X_train)
    #predict = classif.predict(X_train.toarray())
    train_score = float('{0:.3f}'.format(metrics.f1_score(Y_train[:,i], predict)))
    predict = classif.predict(X_test)
    #predict = classif.predict(X_test.toarray())
    test_score = float('{0:.3f}'.format(metrics.f1_score(Y_test[:,i], predict)))
    
    train_score_list.append(train_score)
    test_score_list.append(test_score)
    print(i, 'Detecting {} with {}% f1-score (training {}%)'.format(labels[i],
                                                                 100*test_score,
                                                                 100*train_score))

print()
print('Mean f1 score: testing {}, training {}'.format(np.mean(test_score_list),
                                                     np.mean(train_score_list)))
    
predict_train = np.zeros_like(Y_train)
predict_test = np.zeros_like(Y_test)
for i in range(len(labels)):
    classif = classif_list[i]
    predict_train[:,i] = classif.predict(X_train)
    predict_test[:,i] = classif.predict(X_test)
acc_train = 1 - np.sum(np.abs(predict_train - Y_train))/(Y_train.shape[0]*Y_train.shape[1])
acc_test = 1 - np.sum(np.abs(predict_test - Y_test))/(Y_test.shape[0]*Y_test.shape[1])
print('###')
print('Global accuracy: testing {}, training {}'.format(acc_test, acc_train))

well_labeled = 0
for i in range(len(Y_train)):
    if np.sum(np.abs(Y_train[i,:] - predict_train[i,:])) == 0:
        well_labeled +=1
print('Overall {} out of the {} training samples were well labeled'.format(well_labeled,len(Y_train)))

well_labeled = 0
for i in range(len(Y_test)):
    if np.sum(np.abs(Y_test[i,:] - predict_test[i,:])) == 0:
        well_labeled +=1

print('Overall {} out of the {} testing samples were well labeled'.format(well_labeled,len(Y_test)))

0 Detecting информационные технологии, интернет, телеком with 77.9% f1-score (training 91.0%)
1 Detecting строительство, недвижимость with 72.3% f1-score (training 88.3%)
2 Detecting спортивные клубы, фитнес, салоны красоты with 69.0% f1-score (training 92.60000000000001%)
3 Detecting маркетинг, реклама, pr with 64.4% f1-score (training 90.7%)
4 Detecting медицина, фармацевтика with 77.3% f1-score (training 93.10000000000001%)
5 Detecting консультирование with 39.5% f1-score (training 70.3%)
6 Detecting бухгалтерия, управленческий учет, финансы предприятия with 63.2% f1-score (training 88.1%)
7 Detecting рабочий персонал with 60.0% f1-score (training 82.39999999999999%)
8 Detecting наука, образование with 59.9% f1-score (training 91.9%)
9 Detecting автомобильный бизнес with 59.099999999999994% f1-score (training 87.0%)
10 Detecting управление персоналом, тренинги with 61.4% f1-score (training 92.9%)
11 Detecting транспорт, логистика with 68.89999999999999% f1-score (training 87.8%)
12 

## Try 1.
### 3600 total samples. mean word2vec for document representation. 23 classes

Log reg - Mean f1 score: testing 0.3923809523809524, training 0.4028095238095238

Random forest (default) - Mean f1 score: testing 0.40671428571428575, training 0.9166666666666665

Random forest (n_est=30, max_depth=3) - Mean f1 score: testing 0.48714285714285716, training 0.6254285714285716

Random forest (n_est=30, max_depth=5) - Mean f1 score: testing 0.5288095238095236, training 0.8464285714285713

Random forest (n_est=30, max_depth=7) - Mean f1 score: testing 0.5265238095238095, training 0.9252857142857144

Random forest (n_est=30, max_depth=9) - Mean f1 score: testing 0.4854285714285715, training 0.9510952380952379

Random forest (n_est=40, max_depth=5) - Mean f1 score: testing 0.5297619047619048, training 0.8522380952380951

Random forest (n_est=60, max_depth=5) - Mean f1 score: testing 0.5395714285714286, training 0.8646190476190474

Random forest (n_est=80, max_depth=5) - Mean f1 score: testing 0.5463809523809524, training 0.8645714285714287

Random forest (n_est=100, max_depth=5) - Mean f1 score: testing 0.5424761904761906, training 0.866095238095238 

Random forest (n_est=120, max_depth=5) - Mean f1 score: testing 0.5412857142857144, training 0.8694761904761905

Random forest (n_est=150, max_depth=5) - Mean f1 score: testing 0.5402380952380953, training 0.8681428571428571

GradientBoostingClassifier(n_estimators=80, max_depth=5) - Mean f1 score: testing 0.5571904761904762, training 0.9752380952380952 / time = 3 min

GradientBoostingClassifier(n_estimators=80, max_depth=4) - Mean f1 score: testing 0.5720476190476189, training 0.9719047619047619 

GradientBoostingClassifier(n_estimators=80, max_depth=3)  - Mean f1 score: testing 0.5788095238095239, training 0.9293809523809523 / time = 2 min -----------

GradientBoostingClassifier(n_estimators=80, max_depth=2) - Mean f1 score: testing 0.5351904761904761, training 0.7915238095238095 / time 2 min

GradientBoostingClassifier(n_estimators=30, max_depth=3) - Mean f1 score: testing 0.5093809523809524, training 0.7613333333333333  / time 2 min

GradientBoostingClassifier(n_estimators=100, max_depth=3) - Mean f1 score: testing 0.5757142857142857, training 0.949190476190476

GradientBoostingClassifier(n_estimators=50, max_depth=3) - Mean f1 score: testing 0.5532380952380953, training 0.8664761904761905

GaussianNB() - Mean f1 score: testing 0.35142857142857153, training 0.36628571428571427


## Try 2.
### 10000 total samples. tf-idf for document representation. 20 classes

Log reg - Mean f1 score: testing 0.6833, training 0.8485000000000001 / time=5s ----------

Log reg(weights_for_positive * 0.5) - Mean f1 score: testing 0.70085, training 0.8743000000000001 -----------------

RandomForestClassifier(n_estimators=80, max_depth=5) - Mean f1 score: testing 0.57405, training 0.68235

GaussianNB() - Mean f1 score: testing 0.4888, training 0.8202 / time=2min, long predict time

GradientBoostingClassifier(n_estimators=80, max_depth=3) - Mean f1 score: testing 0.60145, training 0.7172499999999999  / time=8min

### 10000 samples. mean word2vec for document representation

Log rer - Mean f1 score: testing 0.44904999999999984, training 0.45100000000000007

GradientBoostingClassifier(n_estimators=80, max_depth=3) - Mean f1 score: testing 0.6090000000000001, training 0.7715 / time=5min

### 10000 samples. mean word2vec weighted by tf-idf

Lof reg -Mean f1 score: testing 0.37105, training 0.3703

RandomForestClassifier(n_estimators=80, max_depth=5) - Mean f1 score: testing 0.5995000000000001, training 0.7244

GradientBoostingClassifier(n_estimators=80, max_depth=3) - Mean f1 score: testing 0.6052, training 0.7688 /time=5min

## Try 3
### 22000 samples. tfidf.
Log ref - Mean f1 score: testing 0.6993, training 0.8241499999999998

### 22_000 samples. word2ver mean

Log reg  - Mean f1 score: testing 0.46375, training 0.472

GradientBoostingClassifier(n_estimators=80, max_depth=3) - 


## Try 4
### 22000 samples. tf-idf + n_gramms 
#### n_gramms=(1, 2)

Log Reg  - Mean f1 score: testing 0.7274999999999999, training 0.8892999999999999  / 1 min  ----------------

Log reg(weights_for_positive * 0.5) - Mean f1 score: testing 0.7292500000000001, training 0.9058999999999999 / 1 min ----------

Log reg(weights_for_positive * 0.7) -  Mean f1 score: testing 0.73125, training 0.89955 --------------

Log reg(weights_for_positive * 0.8) - Mean f1 score: testing 0.73075, training 0.8967500000000002

#### n_gramms=(1, 3)
Log reg - Mean f1 score: testing 0.72385, training 0.9073499999999999 / 2min

Log reg  (C=0.8)Mean f1 score: testing 0.7222000000000001, training 0.9015000000000001 /2min

Log reg(weights_for_positive * 0.7) - Mean f1 score: testing 0.71845, training 0.915050000000000 /2min

#### n_gramms= (1, 4)
log reg - Mean f1 score: testing 0.71125, training 0.9154000000000002  / 4min

log reg(c=0.9) - Mean f1 score: testing 0.7111000000000001, training 0.91395 /4min

### 25000 samples. tf-idf + n_gramms
####n_gramms=(1, 2)

Log reg - Mean f1 score: testing 0.7184, training 0.8848499999999999 /1min

Log reg(weights_for_positive * 0.5) - Mean f1 score: testing 0.7205, training 0.9047000000000001  /1min

Log reg(weights_for_positive * 0.7) - Mean f1 score: testing 0.7187500000000001, training 0.8970500000000001

Log reg(weights_for_positive * 1.5)  - Mean f1 score: testing 0.70785, training 0.8615

Log reg(weights_for_positive * 1.2)  - Mean f1 score: testing 0.715, training 0.8753500000000001

Log reg(weights_for_positive * 0.9)  -  Mean f1 score: testing 0.7190999999999999, training 0.8894500000000001

#### without n_gramm

Log reg - Mean f1 score: testing 0.6983499999999999, training 0.8177999999999999


#### .....
#### n_gramms=(1, 2)

Log reg - Mean f1 score: testing 0.6512, training 0.89055

Log reg(weights_for_positive * 0.5) - Mean f1 score: testing 0.62905, training 0.9113499999999999